Housekeeping:  
This is code using python code for ArcGIS using the specific arcpy packages    
  
If running in the notebook function within the ArcGIS software, you dont need to do anything special.   
  
If you would like to run in Visual Studio Code:  
Press Control + Shift + P and search for “>Python: Select Interpreter”.  
The command palette will drop down, and you should see some options to seelct. Choose the first option when it says, “Enter interpreter path…”:  
TO find your interpreter path in its default location go to the following path or select the following path in the dropdown menu:  
    C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\  
  
further instructions here: https://resources.esri.ca/getting-technical/how-to-configure-visual-studio-code-with-arcgis-pro-s-python-environment  



# Code Sections  
note - code section links wont work if this cell and the linked cell are not rendered (Cntrl+Enter or Shift+Enter) will render them.  
*There are sections that you will need to go and edit or configure based on your computer and spatial requirements - see chunk below this*      

[Set Workspaces](#workspace)  
[Layers Required](#layersrequired)  
[Layers Created](#layerscreated)  
  
[1. Editing Needed](#editing)    
   > [Setting File Names](#filenames)   
   > [Setting Specific Parameters](#parameters)  
       > Distance to Coast
       > Reef Classifications  
       > Projected Coordinate System  
       > Geographic Coordinate System        
      
[2. Manipulating Downloaded Data](#manipulations)  
   > [Moving layers to gdb folder](#movelayers)  
   > [Allen Atlas Data Management](#reefhab_convert)   
   > [Getting Mangrove Habitat](#mangrove)
   > [Cleaning Population Data](#popconvert)  
   > [Cleaning Sediment Layer](#sed_clean)  
   > [Setting Coordinate Systems](#coordinates)  
   > [Generating Blank Rasters](#blankrasters)   
   
     
[3. Generate Reef Layer](#ReefLayer)    
   >  [3a. Geomorphic + benthic](#allen)  
   >  [3b. Adjacent Reef](#adj_reef)  
   >  [3c. Combine Reef, Adjacent Areas, Mangroves](#combinereeflayers)  
   >  [3d. Scale Reef Later](#scalereef)  
      
[4. Generate Access Layer](#access)  
   >  [4a. Distance from coast](#distcoast)  
   >  [4b. Population estimate to coast](#popcoast)  
   >  [4c. Distance to City](#dist_city)  
   >  [4c. Combine Access Layers](#combineaccess)  
      
[5. Modify Sediment Layer](#sediment)
   >  [5a. Scale Sed Layer](#scale_sed)  
   > this layer isnt explicitly included, but when/if we decide to add sedimentation, its here.

[6. Combine "Impact" Layers](#combine_impact)
   >  [6a. combine, dist, pop, and sediment]
   
  
[7. Combining Reef Area + Access](#impact)    




# Code Sections you *MUST* EDIT   
or at least check to make sure they are correct  
[Set New Geodatabase Folder](#new_gdb)  (only do on the first time running this code)  
[Set Folder Location for Raw Data](#raw_data)  
[Set Workspaces](#workspaces)  
[Setting Layer Names](#filenames)   
[Setting Parameter Values](#parameters)  
[Setting Coordinate Systems](#coordinates)  

# Fishing Impact Workflow

this hazard layer measures the intensity that a resource (in this case, FISH) is used or has the potential to be used, modified, or extracted.  
high hazard values = high intensity use and potential for resournce manipulation  
low hazard values  = low intensity use (or low likelihood or resource use) 
we use the Allen Atlas reef area layer as a proxy for the resource (FISH) 

1. Availability  (of the resournce, fish)  
        > 1a. Marine benthic habitat  
        > 1b. Geomorphic benthic habitat  
        > 1c. Mangrove habitat
        > 1d. Adjacent use areas  
        all layers are scaled, then summed, and then scaled again (0-100)

2. Access  
        > 2a. Distance to coast  
        > 2b. Population  
        > 2c. Distance to major population centers (Provincial Capitals)
        again, all layers are scaled, then combined, then scale again.   

 
3. Combine 'Fishing Access' + 'Fish Resource'   
        > in the forest example we multiplied. So there could ONLY be impact where there was forest.  
        > but the boundaries in marine systems are very different, so there is not a clear boundary between resournce and not (Except for the coast).  so the layers are summed so that the potential for fish exists beyond the boundaries of reef areas.

6. Rescale final 'fishing impact' layer to 0-100  





<a id= "workspace"></a>

# Set Workspaces

1. Set variable names 
2. Put all your downloaded data into one file and load into GIS.  


In [1]:

import arcpy
import os
from arcpy import env



In [2]:
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

## Setting workspaces and folders to save data
<a id= "new_gdb"></a>

In [4]:
#creating a new workspace gdb file to put files into

#keep commented out if you are working on this, but if this is the first time running this code, run the createfilegdb line
#arcpy.management.CreateFileGDB(r"C:\Users\jc446202\OneDrive - James Cook University (1)\ACIAR_spatial\modified_dat", "Fishing_ExPot", "CURRENT")


<Result 'C:\\Users\\jc446202\\OneDrive - James Cook University (1)\\ACIAR_spatial\\modified_dat\\Fishing_ExPot.gdb'>

In [3]:
#create file path directory to where you have saved your downloaded data 
#When running through this code, the new files will save into another folder

##############################################
#YOU NEED TO CHANGE THIS TO FIT YOUR COMPUTER#
##############################################

downloaded_data = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_Impact_Files"
#set your workspace, so everything will save to the new .gdb file you created
forest_gdb = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb"        #or whatever output you want to save your files to. must be a .gdb file - create a .gdb folder.
gdb_workspace = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb"             # r means read the raw file path name and doesnt interpret \ as python syntax fxns

arcpy.env.workspace = gdb_workspace

In [4]:
#check workspace
print(arcpy.env.workspace)
print(downloaded_data)
print(gdb_workspace)

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb
C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_Impact_Files
C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb


<a id= "layersrequired"></a>

# Layers required

**these files should be saved in a folder that only keeps downloaded data**  
set the downloaded data file path in the cell above

Shapefiles should be projected to WGS 1984 PDC Mercator  
Load data to Arc Project (drag and drop from file or Add Data)  
All files are in "Fishing_Impact_Files"  
You should only have *ONE* folder. All these files get put in that folder and will run.  
if you want to expand to another region, or do this for multiple years, those will need to be in separate folders  
<br>


Resource Layers
------------------------

1. Allen Coral Atlas benthic habitats data:  
https://allencoralatlas.org/  
Need to download Benthic map AND geomorphic map for area of interest 
data will download as a geojson  
     - "benthic.geojson"  
     - "geomorphic.geojson"  
     - "ES_benthi.geojson" - the eastern solomons
     - "ES_geomorphic.geojson" - the eastern solomons

2. Mangrove habitat - 2020 v3  
    - "mangrove_shp_SI"
    - from global mangrove watch    
    - https://www.globalmangrovewatch.org/   
    - imported as a shapefile - gmw_v3_2020_vec  
    - this needs to be clipped to the Solomon Island before running this code.   
     
2. Provincial capitals and major cities  
    - this is for distance to city/market which has a negative impact on fish density/abundance/etc.
    - https://solomons.gov.sb/wp-content/uploads/2023/09/  Solomon-Islands-2019-Population-Census-Report_Basic-Tables_Operations_Vol2.pdf  
    - get the gps points of cities from: https://simplemaps.com/data/world-cities  
    - cross reference the census cities, and add them to the gps points   
    - only include cities with >5000 population  
    - file:///C:/Users/jc446202/Downloads/Coral_reef_fish_value_chains_in_Solomon_Islands_Ma.pdf
        - also discusses that there are a few large markets, but that large volumes of fish are sold at small market centers 


3. Major Roads   
    - from open street map - same used in forest layer   
    - https://planet.openstreetmap.org/   
    - https://solomonislands-data.sprep.org/dataset/openstreetmap-data-solomon-islands  
    - "osm_SI_roads_p"  - projected to correct coordinate system


4. Solomon Islands country vector file, projected  
    - name whatever you like, you will change the name later in the code  
    - "Solomons_country_p"   
   
   
5. Solomon Islands country LINE file, projected  
    - name whatever you like, you will change the name later in the code  
    - "Solomons_line"   
  
  
6. Solomons Islands EEZ polygon area, projected  
    - name whatever you like, you will change the name later in the code  
    - "Solomons_EEZ"  
     
     
7. Population count data - SPC  
https://pacificdata.org/data/dataset/slb_population_grid_2020  
Selections:  
Year: 2020  
FileFormat: GeoTiff  
Resolution: 100 m  
    - "slb_rastpop2020rps_100m.tif"   
  
  
8. Enumeration Area polygon data  
https://pacificdata.org/data/geographic_data/2009_slb_phc_admin_boundaries/resource/7b19ab32-2a3b-4afb-84d2-f4deef9050d8  
Enumeration data is the lowest level of administrative boundary available for SI  
    - "SB_2009_EnumArea_4326.shp"   

  
9. Sediment Export Plumes   
https://github.com/WCS-Marine/local-reef-pressures/tree/main/data-raw  
    - Results from this analysis are available as a scientific article: Andrello, M., Darling, E. S., Wenger, A., Suárez-Castro, A. F., Gelfand, S., & Ahmadia, G. N. (2022). A global map of human pressures on tropical coral reefs. Conservation Letters, Early view, e12858.  (https://doi.org/10.1111/conl.12858)  
    - "sed_plume_avg.tif"


  

<a id= "layerscreated"></a>

# Layers Created


## General Layers


| Layer_Name | File_Type | Description |
|    :---    |  :---     |  :---   |
|**country_poly**      | Polygon | country polygon - you set this name in Set Parameters              | 
|**country_EEZ**       | Polygon | EEZ polygon - you set this name in Set Parameters                  |
|**country_line**      | Line    | line out country outline - you set this name in Set Parameters     |
|**Suburb**            | Polygon | Enumeration Area government subdivision areas. smallest govmt area |
|**Suburb_p**          | Polygon | projected Enumeration Area government subdivision areas            |
|**coast_to_20k**      | Polygon | buffer from coastline to 20 km offshore                            |
|**coast_buff_10k**    | Polygon | buffer from coastline to 10 km offshore                            |
|**blank_rast_SI**     | Raster  | blank raster of Solomons EEZ of land and ocean, value = 1          |
|**blank_rast_0**      | Raster  | blank raster of Solomons EEZ of land and ocean, value = 0          |
|**blank_rast_land **  | Raster  | blank raster of Solomons EEZ of land only                          |
|**blank_rast_ocean**  | Raster  | blank raster of Solomons EEZ of ocean only                         |


## Reef Layers

Benthic Layers  

| Benthic Layers | File_Type | Description |
|    :---    |  :---     |  :---   |
|**benthic**                | Polygon   | benthic polygon data converted from geojson from Allen Atlas           |
|**benthic_p**              | Polygon   | polygon file projected into WGS 1984 PDC Mercator                      |
|**benthic_Dissolve**       | Polygon   | geomorphic data dissolved into multipart polys where 1 poly per class  |
|**benthic_Dissolve_p**     | Polygon   | dissolved polygon data projected into WGS 1984 PDC Mercator            |
|**benthic_SI**             | Polygon   | dissolved polygon benthic data clipped to Solomon Islands              |
|**benthic_rast_SI**        | Raster    | dissolved polygon benthic data clipped to Solomon Islands              |
|**benthic_rast**           | Raster    | Raster 50 m cell size of benthic data                                  |
|**benthic_one_poly**       | Polygon   | One polygon of geomorphic layer                                        |
  
 Geomorphic Layers  
   
| Geomorphic Layers | File_Type | Description |
|    :---    |  :---     |  :---   |
|**geomorphic**            | Polygon   | geomorphic polygon data converted from geojson from Allen Atlas         |
|**geomorphic_p**          | Polygon   | polygon file projected into WGS 1984 PDC Mercator                       |
|**geomorphic_Dissolve**   | Polygon   | benthic poly data dissolved into multipart polys of 1 poly per class    |
|**geomorphic_Dissolve_p** | Polygon   | dissolved polygon data projected into WGS 1984 PDC Mercator             |
|**geomorphic_SI**         | Polygon   | dissolved polygon geomorphic data clipped to Solomon Islands            |
|**geomorphic_rast**       | Raster    | Raster 50 m cell size of geomorphic data                                |
|**geomorphic_rast_SI**    | Raster    | Geomorphic rasters of Solomons reefs                                    |
|**geomorphic_one_ploy**   | Polygon   | One polygon of geomorphic layer                                         |


Adjacent Reef Measures  

| Adjacent Reef Layers | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**marine_hab_buff250m**       | Polygon  | buffer of 250 m around marine habitat (marine_hab_one_poly)                     |
|**marine_reef_areas**         | Raster   | Raster (50 m cell size) of reef area (100) and 250 m buffer around reef (50)    |



General Reef Layers

| Layer_Name | File_Type | Description |
|    :---    |  :---     |  :---   |
|**marine_hab**            | Polygon   | marine habitat of Solomons- benthic and geomorphic separate polys       |
|**marine_hab_one_poly**   | Polygon   | marine habitat of benthic and geomorphic as one poly                    |
|**reef_area**            | Polygon   | combined benthic and geomorphic layers with rated values                |
|**reef_area_scale**      | Polygon   | scaled combined benthic and geomprhic layers with raster values         |


Final Resource Layer  
  
| Fish Resource Layer | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**fish_resource_layer**  | Raster   | Final resource layer of reef layer scale + marine reef areas            |
|**fish_resource_scaled** | Raster   | Scaled resource layer, 100 = high resource, 0 = low resource            |


## Sediment Layers


| Sediment Layers | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**sed_plume_avg**               | Raster   | 1 km cell of sediment plume - global coral reefs            |
|**sed_plume_avg_si**            | Raster   | 1 km cell of sediemnt plme in SOls EEZ                      |
|**sed_plume_scale**             | Raster   | 1 km cell of scaled sediemnt plme in SOls EEZ               |



 
## ACCESS LAYERS

Distance Measures  

| Distance Layers | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**Dist_Coast_10k**               | Raster   | 50m cell distance from coastline to 10km offshore                                                   |
|**Dist_Coast_10k**                | Raster   | Distance from coastline both ocean and land to 10000 m. 50 m cell size                              |
|**Offshore_Dist_10k**             | Raster   | Distance from coastline over ocean only. 10000 m with 50m cell size                                 | 
|**dist_from_coast_scale**        | Raster   | Raster (50 m cell size) of benthic layer of Allen Atlas data. values are by fishing preference     | 
|**dist_from_coast_scale_full**   | Raster   | Scaled distance from coast (0 to 100) with 100 = near coast, 0 = far from coast at 5000m           | 


Population Measures  
  
| Population Measures | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**Suburb_pop**                   | Polygon  | Enumeration Area w/population summed to each EA area (Field = Sum)                                 |
|**pop_count_SI**                 | Raster   | raster of population count across Solomons                                                         | 
|**suburb_zonalstats_pop**        | Raster   | Enumeration area zones with total population per EA where pop data came from pop_count_SI          | 
|**Ocean_allocation_pop**         | Raster   | Raster (50 m cell size) where population per ea gets allocated to ocean area                       | 
|**ocean_pop_allo_10k**            | Raster   | Population allocation on ocean areas out to 10000 m (clipped layer from ocean_allocation_pop)       | 
|**ocean_pop_10k_scale**           | Raster   | Scaled ocean allocation. Where sum population values are scaled 0-100. 100 = high pop, 0 = low pop |


Final Access Measures  
  
| Final Access Layers | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**fish_access_layer**            | Raster   | Combined access layer of scaled layers: distance, geomorphic, benthic, ocean_allo_pop              | 
|**fish_access_scale**            | Raster   | Scaled access layer 0-100. 0 = low/no access. 100 = high/most access                               | 

## Final Fish Impact Layer


| Final Fish Impact Layers | File_Type | Description |
|    :---:    |  :---:    |  :---:   |
|**fish_impact**            | Raster   | Combined fish access scale + fish resource scale. Note USE SUM because fishing can occur in all marine areas          | 
|**fish_impact_scale**      | Raster   | Scaled fish impact layers. 0-100. 100 = highest impact. 0 = lowest/no impact                                          | 


<a id= "editing"></a>


# 1. Parameters that need Editing

1. Need to set the file names for some layers
2. Need to set certain parameter values 

### Need to rename the following layers:
- country_polygon = 'Solomons_country_p'           
- country_line = 'solomons_line'
- EEZ = 'Solomons_EEZ_p'
- Suburb = 'SB_2009_EnumArea_4326'

#changing raster name
- pop_count = 'slb_rastpop2020rps_100m.tif'

### Need to set a value for the following parameters:

Listed below are default values, but can be changed if desired  
these values will vary based on expert and informed knowledge about ecological, biophysical and social parameters  

**"offshore_access_distance"**  
    Default =  10000
    this is the distance fishers will travel from the coast to fish. 
    default is 10000 m (10 km)
        
**"cell size"**     
    Default =  50m
    
**adjacent_reef_area**
    Default = 250 m
    this is the distance adjacent to reef locations that fishing is still likely to occur even the its classified as open ocean
    
**project_coord**  
    Default = WGS_1984_PDC_Mercator
    the projects coordinate system to use for this analysis
      
**gcs_coord**  
    Default = GCS_WGS_1984
    the geographic coordinate system. in case you need to convert
    
    


<a id= "filenames"></a>

## Setting file names

In [5]:
#list all files in downloaded data folder to check everything downloaded ok

#set workspace to downloaded data file
arcpy.env.workspace = downloaded_data

#list feature classes
featureclasses_downloaded = arcpy.ListFeatureClasses()
print(featureclasses_downloaded)

#list rasters
downloaded_rasters = arcpy.ListRasters()
print(downloaded_rasters)

#this is a list of the all the files downloaded.

['mangrove_p.shp', 'osm_SI_roads_p.shp', 'SB_2009_EnumArea_4326.shp', 'SI_Cities.shp', 'Solomons_country_p.shp', 'Solomons_EEZ_p.shp', 'solomons_line.shp']
['sed_plume_avg.tif', 'slb_rastpop2020rps_100m.tif']


<a id= "filenames"></a>

## Setting Parameters

In [6]:
# Distance travelled offshore

#offshore_access_distance
offshore_access_distance = 10000  #default is 10000 m


In [7]:
#Adjacent reef area 

adjacent_reef_area = 250   #default is 250 m


In [5]:
#Projected coordinate system

project_coord = arcpy.SpatialReference('WGS_1984_PDC_Mercator')  #default = 'WGS_1984_PDC_Mercator'
project_coord

name (Projected Coordinate System),WGS_1984_PDC_Mercator
factoryCode (WKID),3832
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [9]:
#Geographic coordinate system

gcs_coord = arcpy.SpatialReference('GCS_WGS_1984') #default = 'GCS_WGS_1984'
gcs_coord

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [10]:
print(gdb_workspace)
print(downloaded_data)

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb
C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_Impact_Files


<a id= "manipulations"></a>

# 2. Manipulating downloaded data 

<a id= "movelayers"></a>

## Moving downloaded data to gdb and saving as new file names

In [16]:
for layer in featureclasses_downloaded:
    # Get the full path of the input layer
    input_layer_path = os.path.join(downloaded_data, layer)
    
    # Generate a new name for the output layer
    output_layer_name = arcpy.ValidateTableName(layer, gdb_workspace)
    
    # Set the output layer path in the geodatabase
    output_layer_path = os.path.join(gdb_workspace, output_layer_name)
    
    # Project the layer to the target spatial reference
    arcpy.management.Project(input_layer_path, output_layer_path, project_coord)
    
    print(f"Layer '{layer}' projected and saved to '{output_layer_path}'")

print("Conversion complete.")

Layer 'mangrove_p.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\mangrove_p_shp'
Layer 'mangrove_shp_SI.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\mangrove_shp_SI_shp'
Layer 'mangrove_shp_SI_p.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\mangrove_shp_SI_p_shp'
Layer 'osm_SI_roads_p.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\osm_SI_roads_p_shp'
Layer 'SB_2009_EnumArea_4326.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\SB_2009_EnumArea_4326_shp'
Layer 'SI_Cities.shp' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fis

In [21]:
arcpy.env.workspace = gdb_workspace
gdb_features = arcpy.ListFeatureClasses()
print(gdb_features)

gdb_rasters = arcpy.ListRasters()
print(gdb_rasters)

['mangrove_p_shp', 'osm_SI_roads_p_shp', 'SB_2009_EnumArea_4326_shp', 'SI_Cities_shp', 'Solomons_country_p_shp', 'Solomons_EEZ_p_shp', 'solomons_line_shp']
['sed_plume_avg_tif', 'slb_rastpop2020rps_100m_tif']


In [20]:
#move rasters to geodatabase
#this gets the rasters we want to move, projects them, and then copies them to geodatabase (ExPot)
#
for layer in downloaded_rasters:
    # Get the full path of the input layer
    input_layer_path = os.path.join(downloaded_data, layer)
    
    # Generate a new name for the output layer
    output_layer_name = arcpy.ValidateTableName(layer, gdb_workspace)
    
    # Set the output layer path in the geodatabase
    output_layer_path = os.path.join(gdb_workspace, output_layer_name)
    
    # Project the layer to the target spatial reference
    arcpy.management.ProjectRaster(input_layer_path, output_layer_path, project_coord)
    
    print(f"Layer '{layer}' projected and saved to '{output_layer_path}'")

print("Conversion complete.")

Layer 'sed_plume_avg.tif' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\sed_plume_avg_tif'
Layer 'slb_rastpop2020rps_100m.tif' projected and saved to 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\slb_rastpop2020rps_100m_tif'
Conversion complete.


In [18]:
#Renaming the files in the gdb folder

arcpy.env.workspace = gdb_workspace

#HERE you need to CHANGE THE NAMES OF THE FILES on the right of the = sign 
#using the names of the files you have downloaded
#only need to change the below files

#naming the files so can call them easily later (note - does NOT acutally change the name, just makes it an object)
country_polygon = 'Solomons_country_p_shp'            #this is where you will change the name of the shapefile you have downloaded. 
country_line = 'solomons_line_shp'
EEZ = 'Solomons_EEZ_p_shp'
Suburb = 'SB_2009_EnumArea_4326_shp'
roads = 'osm_SI_roads_p_shp'
cities = 'SI_Cities_shp'




<a id= "reefhab_convert"></a>


## Allen Atlas Data Management

1. need to move the geojson file from downloaded data and turn in to polygon file in geodatabase folder  
2. dissolve the data. the polygon files have >400,000 objects, and want to combine all polygons by type so its more manageable to process  
3. be aware this is a computational intensive process. will take time and make sure not to run again if going through this code more than once.  

In [23]:
arcpy.env.workspace = downloaded_data
print(downloaded_data)

#list all allen atlas data in the downloaded data file (fishing_impact_files)
Allen_List = arcpy.ListFiles("*geojson") # * is a wild cards. says select anything that has "...geojson" and any value before that.
print(Allen_List)
print(len(Allen_List))

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_Impact_Files
['benthic.geojson', 'geomorphic.geojson']
2


In [19]:
#renames files and converts to polygon into downloaded data file
#keeping as commented right now, because it takes a long while to run and dont want to accidentally run twice


#for f in arcpy.ListFiles('*.geojson'):                                                 # lists all geojsons in the active workspace (downloaded data)
    #root_extension = os.path.splitext(f)                                               # separates root and extension of file names
    #output_poly = os.path.join(os.path.dirname(f), root_extension[0])                  # creates new list of file directory and root names of geojson files
    #print("Processing : {}".format(output_poly))                                       # lets you know which file is being processed
    
    #arcpy.conversion.JSONToFeatures(f, output_poly)                                    # converts geojson to poly and store in same directory
    
    #arcpy.management.Project(output_poly, output_poly, project_coord)                  #project the feature class to the specific coordinate system PDC mercator

    #dissolved_layer = os.path.join(gdb_workspace, root_extension[0] + "_dissolve")    # assign layer name to identify layers to dissolve
    #arcpy.management.Dissolve(output_poly, dissolve_layer, "class", None, "MULTI_PART")#dissolve layer by class

Processing : D:\ACIAR\modified_dat\Fishing_Impact.gdb\benthic
Processing : D:\ACIAR\modified_dat\Fishing_Impact.gdb\geomorphic


In [20]:
#repair geometries
#for some reason the polygons might have weird geometries and need repairing (e.g. self intersections)
#arcpy.management.RepairGeometry("benthic", "DELETE_NULL", "ESRI")
#arcpy.management.RepairGeometry("geomorphic", "DELETE_NULL", "ESRI")

#might not be necessary, and if it isnt, then it wont modify the input. Does take some time. so dont run unless the next cell chunk doesnt run


<Result 'geomorphic'>

###  Clip marine hab layers to Solomons Country

In [45]:
#BENTHIC CLIP

arcpy.analysis.Clip('benthic_dissolve', 'country_EEZ', "benthic_SI")
arcpy.management.Delete('benthic_dissolve)')



<Result 'D:\\ACIAR\\modified_dat\\Fishing_Impact.gdb\\benthic_SI'>

In [46]:
#GEOMORPHIC CLIP

arcpy.analysis.Clip('geomorphic_dissolve', 'country_EEZ', "geomorphic_SI")
arcpy.management.Delete('geomorphic_dissolve)')


<Result 'D:\\ACIAR\\modified_dat\\Fishing_Impact.gdb\\geomorphic_SI'>

## Erase mangrove from Land Area  

want to have mangrove NOT in land areas  
doing this for a few reasons:  
    1. Already have mangrove in forest estimations  - so dont want to double up mangrove effect   
    2. Mangroves in the indo pacific have slightly less tidal range than other areas (Igulu) and perhaps just having the "marine" non land mangrove area is enough  
    3. But IF I HAVE TIME - i should go through some literature to see if i need to include the 'land' mangrove areas.   

In [32]:
#MANGROVE ERASE

arcpy.analysis.Erase('mangrove_p_shp', 'country_poly', "Mangrove_p_Erase")
arcpy.management.Delete('mangrove_p_shp')


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_Impact.gdb\\Mangrove_p_Erase'>

<a id= "popconvert"></a>


## Population Manipulation

1. need to clip the data to solomons only area
2. when clip, save it to gdb folder

In [24]:
#clip raster data to only area of Solomons 
#and save it to the geodatabase

arcpy.env.workspace = gdb_workspace

out_raster = arcpy.sa.ExtractByMask('slb_rastpop2020rps_100m_tif', EEZ, "INSIDE"); out_raster.save(os.path.join(gdb_workspace, "pop_count_SI"))
#os.path.join means its saving the raster to the gdb workspace and making the file name pop_count_SI

arcpy.management.Delete('slb_rastpop2020rps_100m_tif')                      

<Result 'true'>

<a id= "sed_clean"></a>

## Cleaning sediment layer

In [26]:
#clip sediment layer to solomon islands EEZ

arcpy.env.workspace = gdb_workspace
arcpy.management.Clip("sed_plume_avg_tif", EEZ, "sed_plume_SI")
arcpy.management.Delete('sed_plume_avg_tif')            #remove original sediment layer          

<Result 'true'>

In [27]:
print(arcpy.env.workspace)
arcpy.ListRasters()

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb


['pop_count_SI', 'sed_plume_SI']

<a id= "coordinates"></a>


## Setting Coordinate References and Converting to Projection if needed

In [11]:
#check projection reference - make sure this is correct.

project_coord

gcs_coord 


name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


### coordinates for vector files

In [29]:
#for vector files: check spatial reference

#make sure youre in the right workspace (gdb file)
arcpy.env.workspace = gdb_workspace

#list vectors in gdb
feature_class_gdb = arcpy.ListFeatureClasses()

#Loop through the list
for fc in feature_class_gdb:
    # Create the spatial reference object
    spatial_ref = arcpy.Describe(fc).spatialReference

    # If the spatial reference is unknown
    if spatial_ref.name == "Unknown":
        print("{} has an unknown spatial reference".format(fc))

    # Otherwise, print out the feature class name and spatial reference
    else:
        print("{} : {}".format(fc, spatial_ref.name))

mangrove_p_shp : WGS_1984_PDC_Mercator
osm_SI_roads_p_shp : WGS_1984_PDC_Mercator
SB_2009_EnumArea_4326_shp : WGS_1984_PDC_Mercator
SI_Cities_shp : WGS_1984_PDC_Mercator
Solomons_country_p_shp : WGS_1984_PDC_Mercator
Solomons_EEZ_p_shp : WGS_1984_PDC_Mercator
solomons_line_shp : WGS_1984_PDC_Mercator
benthic_dissolve : WGS_1984_PDC_Mercator
geomorphic_dissolve : WGS_1984_PDC_Mercator


In [30]:
#convert non projected data to projected data (PDC mercator)

#Loop through the list
for infc in feature_class_gdb:                                            
    #determine if input has a defined coordinate system
    spatial_ref = arcpy.Describe(infc).spatialReference

    # If the spatial reference is not the correct projected coordinate system
    if spatial_ref.name == project_coord.name:
        print("{} is already in correct projection".format(infc))
    
    else:
        # determine new output feature class (outfc) path and name. 
        #this saves to same gdb, saves as same name, and then add "_p" to the end to indicate its projected
        outfc = os.path.join(gdb_workspace, infc+'_p')
        
        # set projected coord system to use
        outCS = project_coord
        
        #run project tool
        arcpy.management.Project(infc, outfc, outCS)
        print("{} has been  converted and is now: {}".format(infc, outfc))




mangrove_p_shp is already in correct projection
osm_SI_roads_p_shp is already in correct projection
SB_2009_EnumArea_4326_shp is already in correct projection
SI_Cities_shp is already in correct projection
Solomons_country_p_shp is already in correct projection
Solomons_EEZ_p_shp is already in correct projection
solomons_line_shp is already in correct projection
benthic_dissolve is already in correct projection
geomorphic_dissolve is already in correct projection


### coordinates for raster files

In [31]:
#check spatial reference of raster  files:

#make sure youre in the right workspace (gdb file)
arcpy.env.workspace = gdb_workspace


#list rasters in gdb
raster_gdb = arcpy.ListRasters()
print(raster_gdb)

#Loop through the list
for fc in raster_gdb:
    # Create the spatial reference object
    spatial_ref_rast = arcpy.Describe(fc).spatialReference

    # If the spatial reference is unknown
    if spatial_ref_rast.name == "Unknown":
        print("{} has an unknown spatial reference".format(fc))

    # Otherwise, print out the feature class name and spatial reference
    else:
        print("{} : {}".format(fc, spatial_ref_rast.name))

['pop_count_SI', 'sed_plume_SI']
pop_count_SI : WGS_1984_PDC_Mercator
sed_plume_SI : WGS_1984_PDC_Mercator


In [32]:
#convert non projected data to projected data (PDC mercator)

#Loop through the list
for infc in raster_gdb:                                            
    #determine if input has a defined coordinate system
    spatial_ref = arcpy.Describe(infc).spatialReference

    # If the spatial reference is not the correct projected coordinate system
    if spatial_ref.name == project_coord.name:
        print("{} is already in correct projection".format(infc))
    
    else:
        # determine new output feature class (outfc) path and name. 
        #this saves to same gdb, saves as same name, and then add "_p" to the end to indicate its projected
        outfc = os.path.join(gdb_workspace, infc+'_p')
        
        # set projected coord system to use
        outCS = project_coord
        
        #run project tool
        arcpy.management.ProjectRaster(infc, outfc, outCS)
        print("{} has been converted and is now: {}".format(infc, outfc))



pop_count_SI is already in correct projection
sed_plume_SI is already in correct projection


<a id= "blankrasters"></a>

## Generating Blank Rasters

1. create full blank raster (value = 0) across entirety of Solomons EEZ
2. Create ocean blank raster (no cells for land areas)
3. create land balnk raster (no cells for ocean area) --- tbd on whether i need this one or not. probs for the land based one? to have adj ares and roads become full raster area. 

In [33]:
## Generating Blank raster of Solomons
arcpy.conversion.PolygonToRaster(EEZ, "OBJECTID", "blank_rast_SI", "CELL_CENTER", "NONE", 50, "BUILD")
#turn value of blank raster into 0 (changes from 1 to 0)
arcpy.ddd.Reclassify("blank_rast_SI", "Value", "1 0", "blank_rast_0", "DATA")

<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\blank_rast_0'>

In [13]:
## Generating a Blank raster of land 
arcpy.conversion.PolygonToRaster(country_polygon, "OBJECTID", "blank_rast_land", "CELL_CENTER", "NONE", 50, "BUILD")


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\blank_rast_land'>

In [15]:
#clip raster to ocean area
out_raster = arcpy.sa.ExtractByMask("blank_rast_0", country_polygon, "OUTSIDE"); out_raster.save("blank_rast_ocean")

<a id= "ReefLayer"></a>


# 3. Availability (Reef Area)  

**a. benthic layer + geomorphic layer edits  (Reef layer)  
  b. adjacent to habitat layer  (Fish layer)  
  c. mangrove layer  
  c. combine reef and fish layers**  
  
  
i.    edit benthic & geomorphic layer so hab types are classified into values  
ii.   clip polygon layer to Solomons Islands country layer  
iii.  assign value to mangrove layer  = 2  
iii.  convert benthic, geomorphic, and mangrove layers into raster and clip to solomons area 
iv.   combine benthic, geormorphic, and magrove rasters  
v.    scale marine habitat layer raster 0-100 ("reef area")  
  
vi.    create one polygon for all reef areas  
vii.   create buffer of reef area = 250 m  
viii.  turn buffer area into raster with value of 100= reef, 50 = adjacent buffer, 0 outside both  
  
ix.    combine reef area scaled layer with adjacent reef layer  
x.     scale full layer (100-0)  


<a id= "allen"></a>

## 3A. Reef layer edits

In [43]:
#BENTHIC HABITAT

#add new field called class_num
arcpy.management.AddField('benthic_SI', "Class_Num", "SHORT")

#update Class num field
fc = 'benthic_SI'
fields = ['class', 'Class_Num']

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    #for each row, evaluate the "class" field value (index position of 0)
    #and update "Class_Num" field value (index position of 1 in fields df)
    for row in cursor:
        if (row[0] == "Coral/Algae"):
            row[1] = "3"
        elif (row[0] == "Microalgal Mats"):
            row[1] = "2"
        elif (row[0] == "Rock"):
            row[1] = "1"
        elif (row[0] == "Rubble"):
            row[1] = "1"
        elif (row[0] == "Sand"):
            row[1] = "1"
        elif (row[0] == "Seagrass"):
            row[1] = "2"      
        #update cursor with the updated list
        cursor.updateRow(row)

In [44]:
# GEOMORPHIC HABITAT

#add new field called class_num
arcpy.management.AddField('geomorphic_SI', "Class_Num", "SHORT")

fc = 'geomorphic_SI'
fields = ['class', 'Class_Num']

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    #for each row, evaluate the "class" field value (index position of 0)
    #and update "Class_Num" field value (index position of 1 in fields df)
    for row in cursor:
        if (row[0] == "Back Reef Slope"):
            row[1] = "2"
        elif (row[0] == "Deep Lagoon"):
            row[1] = "2"
        elif (row[0] == "Inner Reef Flat"):
            row[1] = "1"
        elif (row[0] == "Outer Reef Flat"):
            row[1] = "1"
        elif (row[0] == "Plateau"):
            row[1] = "2"
        elif (row[0] == "Reef Crest"):
            row[1] = "3" 
        elif (row[0] == "Reef Slope"):
            row[1] = "3" 
        elif (row[0] == "Shallow Lagoon"):
            row[1] = "2" 
        elif (row[0] == "Sheltered Reef Slope"):
            row[1] = "3"      
        elif (row[0] == "Terrestrial Reef Flat"):
            row[1] = "1"             
        #update cursor with the updated list
        cursor.updateRow(row)
        


In [ ]:
#MANGROVE HABITAT

#add new field called class_num
arcpy.management.AddField(Mangrove, "Class_Num", "SHORT")

#update Class num field
fc = Mangrove
field = ['Class_Num']


#Set mangrove area value to 2

with arcpy.da.UpdateCursor(fc, field) as cursor:
    for row in cursor:
        if row[0] is None:  # Check for None values explicitly
            row[0] = 2
            cursor.updateRow(row)  # Update the row
print("Field 'Class Num' has been added and updated with the value of 2")

### Convert to Raster

In [47]:
#convert to raster - benthic
#cell size = 50 m

arcpy.conversion.PolygonToRaster("benthic_SI", "Class_Num", "benthic_rast", "CELL_CENTER", "NONE", 50, "BUILD")
#note, this takes about 1 hour to complete 

<Result 'D:\\ACIAR\\modified_dat\\Fishing_Impact.gdb\\benthic_rast'>

In [49]:
#convert to raster = geomorphic
#cell size = 50 m

arcpy.conversion.PolygonToRaster("geomorphic_SI", "Class_Num", "geomorphic_rast", "CELL_CENTER", "NONE", 50, "BUILD")


<Result 'D:\\ACIAR\\modified_dat\\Fishing_Impact.gdb\\geomorphic_rast'>

In [34]:
#convert to raster = mangrove
#cell size = 50 m

arcpy.conversion.PolygonToRaster("Mangrove_p_Erase", "Class_Num", "mangrove_rast", "CELL_CENTER", "NONE", 50, "BUILD")


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_Impact.gdb\\mangrove_rast'>

In [22]:
print(gdb_workspace)

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb


### Combine Benthic, Geomorphic, and Mangrove Rasters

In [23]:
#Adding rasters together, but also adding with blank raster to create a full coverage
out_raster = arcpy.ia.CellStatistics("blank_rast_ocean;geomorphic_rast;benthic_rast;mangrove_rast","SUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT");
out_raster.save("marine_hab")


### Scale Fishing Resource Raster

1. extract statistics data to scale values
2. scale values


In [24]:
#identify min and max values of fishing_resource

reef_area_min_output = arcpy.management.GetRasterProperties('marine_hab', 'MINIMUM',"") #extracts minimum value from raster
reef_area_min = int(reef_area_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(reef_area_min)

reef_area_max_ouput = arcpy.management.GetRasterProperties('marine_hab', 'MAXIMUM',"")
reef_area_max = int(reef_area_max_ouput.getOutput(0))
print(reef_area_max)

0
8


In [25]:
#checking object type to confirm integer
print(type(reef_area_min))
print(type(reef_area_max))

<class 'int'>
<class 'int'>


In [27]:
#scale reef layer raster from 0 - 100 

outraster = ((arcpy.Raster("marine_hab") - float(reef_area_min))/(float(reef_area_max) - float(reef_area_min)))*100; outraster.save("marine_hab_scale")


<a id= "adj_reef"></a>

## 3b. Adjacent to Reef Area

want to create a layer showing some fishing effort in the adjacent reef area
remember the resource is FISH  
so adjacent to the reef area is still a location where there are fish, and people are fishing,  
the hazard of extraction of the fish resource is possible. 

Adjacent reef area cutoff distance = 250 m. 

fish area
    reef = 100
    adjacent areas = 50
    aka reef areas are twice as likely to have a fish resource compared to adjacent areas

when we combine fish layer with the reef_layer (benthic and geomorphic layers classed by high fish areas) (100-0)
    we get max possible value for a reef area to be 200, and the adjacent area = 50. 
    meaning that the adjacent area is 25% as good as a high fish reef area (coral reef slope)
    and potentially as good for fish as a low value fish area (rubble reef flat)

steps:
1. dissolve all poly of geo and benthic.
2. merge benthic and geo, and dissolve
3. create 250 buff outside polys
4. merge reef poly and buff poly
5. reef poly = 100, buff poly = 50
6. convert reef+adj poly into raster

7. combine rast with reef_layer
8. scale combined layer (100-0)

In [28]:
#1. dissolve all poly of geo and benthic
arcpy.management.Dissolve("geomorphic_SI", "geomorphic_onepoly", "", None, "MULTI_PART", "DISSOLVE_LINES", '')
arcpy.management.Dissolve("benthic_SI", "benthic_onepoly", "", None, "MULTI_PART", "DISSOLVE_LINES", '')
arcpy.management.Dissolve("Mangrove_p_Erase", "mangrove_onepoly", "", None, "MULTI_PART", "DISSOLVE_LINES", '')



<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\mangrove_onepoly'>

In [88]:
#Delete mangrove_p_Erase

arcpy.management.Delete("mangrove_p_Erase")

<Result 'true'>

In [32]:
#2. merge benthic, geomorphic, and mangrove, and dissolve
arcpy.management.Merge("benthic_onepoly;geomorphic_onepoly;mangrove_onepoly", "marine_habitats", None, "NO_SOURCE_INFO")

#dissolve all polygons so get one large polygon of all marine habitats
arcpy.management.Dissolve("marine_habitats", "marine_hab_onepoly", None, None, "MULTI_PART", "DISSOLVE_LINES", '')

#delete intermediate layer
arcpy.management.Delete("marine_habitats")


<Result 'true'>

In [33]:
#3. create 250 buff outside polys
#dissolved into only polygon

arcpy.analysis.PairwiseBuffer("marine_hab_onepoly", "marine_hab_buff250m", "250 Meters", "ALL", None, "PLANAR", "0 Meters")

print("Pairwise buffer analysis complete.")

Pairwise buffer analysis complete.


In [37]:
#4. erase buffer areas from land, so only on ocean
arcpy.analysis.Erase("marine_hab_buff250m", country_polygon, "marine_hab_buff250m_ocean", None)

#4a. erase reef areas from buffer area - so the buffer areas are only outside of the 
arcpy.analysis.Erase("marine_hab_buff250m_ocean", "marine_hab_onepoly", "marine_hab_buff250", None)

#4c. delete intermediate files 
arcpy.management.Delete("marine_hab_buff250m_ocean")
arcpy.management.Delete("marine_hab_buff250m")

<Result 'true'>

In [38]:
#5a. add field for reef poly = 100, 

#create new field in attribute table called VALUE (to assign value to reef area)
arcpy.management.AddField('marine_hab_onepoly',"Value", 'SHORT',3,"","","Value","","","")

#Set reef area value to 100
fc = "marine_hab_onepoly"
field = ['Value']

with arcpy.da.UpdateCursor(fc, field) as cursor:
	for row in cursor:
        	if row[0] == None: 
            		row[0] = 100
            		cursor.updateRow(row) #NOTE INDENTS ARE VERY IMPORTANT
      


In [39]:
#5a. add field for buff poly = 50, 

#create new field in attribute table called VALUE (to assign value to reef area)
arcpy.management.AddField('marine_hab_buff250',"Value", 'SHORT',3,"","","Value","","","")

#Set reef area value to 100
fc = "marine_hab_buff250"
field = ['Value']

with arcpy.da.UpdateCursor(fc, field) as cursor:
	for row in cursor:
        	if row[0] == None: 
            		row[0] = 50
            		cursor.updateRow(row) #NOTE INDENTS ARE VERY IMPORTANT

In [40]:
#6. combine reef poly and buff poly
#for some reason merge is not working with these layers. must be very very small topology issues that limit data transfer of the value field when merge
#instead - make a feature class 
#then APPEND the two layers together and save into new feature class created
#value = 100 is marine habitat (benthic, geomorphic, mangrove)
#value = 50 is areas within 250 m of marine habitat

arcpy.management.CreateFeatureclass(gdb_workspace, "marine_and_adjacent_hab", 'POLYGON', 'marine_hab_buff250',"", "", project_coord)
arcpy.management.Append("marine_hab_onepoly;marine_hab_buff250", "marine_and_adjacent_hab", "TEST", None, '', '')

<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\marine_and_adjacent_hab'>

In [42]:
#7. convert reef+adj poly into raster

with arcpy.EnvManager(cellSize="fish_access_scale"):
    arcpy.conversion.PolygonToRaster("marine_and_adjacent_hab", "Value", "marine_and_adjacent_hab_rast", "CELL_CENTER", "NONE", 50, "BUILD")

Start Time: Thursday, 22 September 2022 12:28:49 PM
Succeeded at Thursday, 22 September 2022 2:38:04 PM 
(Elapsed Time: 2 hours 9 minutes 14 seconds)

 <a id= "combinereeflayers"></a>

## 3c. Combine Reef and Fish Layers

In [43]:
#combine layers
#marine_fishing_areas_raster: Reef Area = 100, 250 buff around reef areas = 50
#reef_area_scale: 100-0 scaled values for benthic and geomorphic habitat types

out_raster = arcpy.ia.CellStatistics("marine_and_adjacent_hab_rast;marine_hab_scale","SUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT");
out_raster.save("fish_resource_layer")



In [44]:
#scale final layer
#identify min and max values of fishing_resource

fish_res_min_output = arcpy.management.GetRasterProperties('fish_resource_layer', 'MINIMUM',"") #extracts minimum value from raster
fish_res_min = int(fish_res_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(fish_res_min)

fish_res_max_ouput = arcpy.management.GetRasterProperties('fish_resource_layer', 'MAXIMUM',"")
fish_res_max = int(fish_res_max_ouput.getOutput(0))
print(fish_res_max)




0
200


In [45]:
#checking object type to confirm integer
print(type(fish_res_min))
print(type(fish_res_max))

<class 'int'>
<class 'int'>


In [46]:
#scale fishing resource raster from 0 - 100 

outraster = ((arcpy.Raster("fish_resource_layer") - fish_res_min)/(fish_res_max - fish_res_min))*100; outraster.save("fish_resource_scale")

<a id= "access"></a>


# 4. Access Layer

a. Distance to coast
b. population extrapolated to ocean area
c. combine distance and ocean pop and scale




<a id= "distcoast"></a>

## 4a. Distance to coast

a. create buffer area 
    - distance = 10 km
b. *euclidian distance from coastline*
c. scale distance to 0-100. 100 = near coast, 0 = far from coast

In [47]:
#a. create a buffer area from coastline to 20km offshore. 
#just allows to clip,select etc
#outside only means doesnt include land area
arcpy.analysis.Buffer(country_polygon, "coast_to_20k", "20 Kilometers", "OUTSIDE_ONLY", "ROUND", "ALL", None, "PLANAR")

<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\coast_to_20k'>

In [48]:
print(offshore_access_distance)

10000


In [49]:
#b. create distance measure raster from coast
#distance is 10 km

# distance called offshore_access_10k (10000 is default )
out_distance_raster = arcpy.sa.EucDistance(country_line, offshore_access_distance, 50, None, "PLANAR", None, None); out_distance_raster.save(os.path.join(gdb_workspace, 'Dist_Coast_10k'))        

In [50]:
#clip raster distance layer to only offshore areas 
#bc the distance measures either side of the country line (i.e. ocean and land)
out_raster = arcpy.sa.ExtractByMask("Dist_Coast_10k", "coast_to_20k", "INSIDE"); out_raster.save("Dist_offshore_10k")

#delete intermediate raster
arcpy.management.Delete("Dist_Coast_10k")

<Result 'true'>

In [51]:
#c. scale distance from coast

#identify min and max values of distance from coast

coast_dist_min_output = arcpy.management.GetRasterProperties('Dist_offshore_10k', 'MINIMUM',"") #extracts minimum value from raster
coast_dist_min = int(coast_dist_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(coast_dist_min)

coast_dist_max_ouput = arcpy.management.GetRasterProperties('Dist_offshore_10k', 'MAXIMUM',"")
coast_dist_max = int(coast_dist_max_ouput.getOutput(0))
print(coast_dist_max)

0
10000


In [52]:
#checking object type to confirm integer
print(type(coast_dist_min))
print(type(coast_dist_max))

<class 'int'>
<class 'int'>


In [54]:
# scale distance raster 0 -100


#have to subtract by 100 and take the absolute value so invert the distance from coast, so the high values in the scaled layer are near the coast, and 0 is far from the coast
outraster = arcpy.sa.Abs((((arcpy.Raster("Dist_offshore_10k") - coast_dist_min)/(coast_dist_max - coast_dist_min))*100)-100); 
outraster.save("dist_from_coast_scale")

#otherwise you can run this rescale by function, but it does the same thing.
#out_raster = arcpy.sa.RescaleByFunction("Offshore_Dist_5k", "MSSMALL # # # # # #", 100, 0); out_raster.save("dist_coast_rescalefxn")


In [56]:
# combine distance raster with blank raster to get full coverage across all solomons

out_raster = arcpy.ia.CellStatistics("blank_rast_ocean;dist_from_coast_scale","SUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT");
out_raster.save("dist_from_coast_scale_SI")

#delete intermediate layer
arcpy.management.Delete("dist_from_coast_scale")

<Result 'true'>

<a id= "popcoast"></a>

## 4b. Population est to coast

a. summarise  raster values to enumeration area suburb polygon data. zonal statistics as table  
b. join sum zone table to suburb layer  
c. extrapolate total pop counts to coastal areas. 
d. clip pop extrapolation raster to 5k from shore
e. scale extrapolated population raster

In [57]:
#a. summarise raster population data to enumeration areas (suburb) layer and output as a table.
arcpy.ia.ZonalStatisticsAsTable(Suburb, "EAID", "pop_count_SI", "EnumArea_totalpop_table", "DATA", "SUM")



<geoprocessing server result object at 0x212f06e9570>

In [79]:

#arcpy.management.Delete('Suburb_pop')
print(Suburb)

SB_2009_EnumArea_4326_shp


In [78]:
#b. create new suburb layer so we can keep original unedited
#arcpy.management.CopyFeatures(Suburb, "Suburb_pop")

#join table of summed pop to suburb layer
arcpy.management.AddJoin(
    in_layer_or_view="Suburb_pop",
    in_field="EAID",
    join_table="EnumArea_totalpop_table",
    join_field="EAID",
    join_type="KEEP_ALL",
    index_join_fields="INDEX_JOIN_FIELDS"
)


<Result 'Suburb_pop_Layer7'>

In [66]:
#make a summarise raster zone layer
out_raster = arcpy.ia.ZonalStatistics(Suburb, "EAID", "pop_count_SI", "SUM", "DATA", "CURRENT_SLICE", 90, "AUTO_DETECT", "ARITHMETIC", 360); 
out_raster.save("suburb_zonalstats_pop")

In [67]:
#add new field in attribute table to rename field and make as integer
arcpy.management.AddField('Suburb_pop', "Pop_Sum", "SHORT")

#copy to new field as save as integer
#field is called "Suburb_pop.Pop_Sum" because of the way the add join combined fields in a layer
#arcpy.management.CalculateField('Suburb_pop', 'Suburb_pop.Pop_Sum', '!EnumArea_totalpop_table.SUM!', 'PYTHON3','', 'SHORT')


#NOTE SOMETHING WEIRD HAPPENING HERE! need to go back through this code. 

<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\Suburb_pop'>

In [81]:
#c. extrapolate total pop to ocean areas using euclidian distance
#parameters:
    # max distance = 5000 m
    # cell size = the same as the blank rast ocean (50m)
    # allocation value = summed population count for each suburb/enumeration area
with arcpy.EnvManager(cellSize="blank_rast_ocean"):
    out_distance_allocation_raster = arcpy.sa.DistanceAllocation("Suburb_pop", None, None, None, None, "BINARY 1 -30 30", None, "BINARY 1 45", None, None, None, None, "Pop_Sum", None, None, None, '', "PLANAR"); 
    out_distance_allocation_raster.save("Ocean_allocation_pop")

In [82]:
#d. clip extrapolation raster (ocean allocation) to 10km coastline buffer

#must create 5k coast buffer  first. 
arcpy.analysis.Buffer(country_polygon, "coast_buff_10k", "10 Kilometers", "OUTSIDE_ONLY", "ROUND", "ALL", None, "PLANAR")


<Result 'C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\\coast_buff_10k'>

In [83]:
# extract by mask (clip) raster data into 5k buff area
out_raster = arcpy.sa.ExtractByMask("Ocean_allocation_pop", "coast_buff_10k", "INSIDE"); out_raster.save("ocean_pop_10k")



In [85]:
#e. scale distance from coast

#identify min and max values of ocean pop allocation layer

ocean_pop_min_output = arcpy.management.GetRasterProperties('ocean_pop_10k', 'MINIMUM',"") #extracts minimum value from raster
ocean_pop_min = int(ocean_pop_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(ocean_pop_min)

ocean_pop_max_ouput = arcpy.management.GetRasterProperties('ocean_pop_10k', 'MAXIMUM',"")
ocean_pop_max = int(ocean_pop_max_ouput.getOutput(0))
print(ocean_pop_max)

30
2748


In [87]:
#e. scale ocean population raster

outraster = ((arcpy.Raster("ocean_pop_10k") - ocean_pop_min)/(ocean_pop_max - ocean_pop_min))*100; outraster.save("ocean_pop_10k_scale")



In [89]:
#delete intermediate layers
arcpy.management.Delete("Ocean_allocation_pop")
arcpy.management.Delete("EnumArea_totalpop_table")


<Result 'true'>

<a id= "dist_city"></a>

## 4c. Distance to Major Cities  - Gravity

Im going to attempt the gravity function. or a version of it  

Cinner does it for many reef site locations  
    - by clipping populations cells within 500 km of reef sites  
    - measure least cost path using some R function of land cover to measure least cost distance between two cells   
    - dist measures from every populated cell on land to centroid of nearest reef site  (Dist from cell to reef point)  
    - gravity is then measured as:  
            - pop of cell / (travel time between reef site and that cell)^2  
            - summed the gravity values for every cell within 500 km of reef sites to get total gravity for a reef area  
  
  
Im going to:  
    - measure the distance from reef cell to population point (market/city locations)
    - from the city center point (cell) outwards by distance.  
    - weight the travel time by the nearest city population. 
    - using the tool: " tbd "
      
  
we already have:  
    - euclidian distance from coast - linear decay  
    - population of coastal municipality projected into water areas  
    - those combined give the scaled allocation of population onto marine areas   
            - I am posing that this is an estimate of SUBSISTENCE fishing.   
    - 
what the distance to market would add  (travel time weighted by population of city)  
    - is some measure of more commercialised (but not that word) fishing where markets would draw extra fishing effort.   
    - we are not using it as a FISH BIOMASS estimator like what cinner was doing  
    - we are using it as a pressure indicator. kind of the inverse of what cinner et al did.  
    - still want to make a least cost path informed by cinners travel time estimates:  
    - Need to add road to raster layer  
  
| ESA Land Cover Layer   | Value   | Speed (km/h) |  Time (min/km) |
|    :---                |  :---   | :---         |  :---          |
|Tree Cover              |  10     |    1     |   60    | 
|Shrubland               |  20     |    1.6   |   37.5  |
|Grassland               |  30     |    1.6   |   37.5  |
|Cropland                |  40     |    1.6   |   37.5  |
|Built Up                |  50     |    30    |   2     |
|Bare/Sparse Vegetation  |  60     |    1.25  |   48    |
|Snow and Ice            |  70     |    1     |   60    |
|Permanent Water Bodies  |  80     |    20    |   3     |
|Herbaceous Wetland      |  90     |    1     |   60    |
|Mangrove                |  95     |    1     |   60    |
|Road                    |  100    |    60    |   1     |
*road gets added to raster layer  


Steps: 
1. convert road to raster layer (same resolution as ESA)
    - road value = 100
2. merge road rater and ESA layer together where road takes precedence over ESA layer
3. reclassify esa and road raster layer to above classifications for *TIME* - so faster travel = lower cost
4. calcuate least cost path distance. 
5. or gravity = meaning travel time from each market area (within each cell) (pop of market city/(cell value of tt)^2)

In [91]:
#1. convert road to raster

#1a. add field to attribute table and assign value of 100

arcpy.env.workspace = gdb_workspace

roads = "osm_SI_roads_p_shp"

new_field = "Value"

#add new field to attribute table
arcpy.AddField_management(roads, new_field, "SHORT")

with arcpy.da.UpdateCursor(roads, new_field) as cursor:
    for row in cursor:
        row[0] = 100
        cursor.updateRow(row)

print("Field '{}' added to the attribute table and all values now 100". format(new_field))


Field 'Value' added to the attribute table and all values now 100


In [92]:
#1b convert to raster
roads = "osm_SI_roads_p_shp"

output_raster = "road_rast"

field = "Value"

#cell size reference to the ESA layer so they match resolution
esa_raster = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb/esa2020_si"

#Obtain the cell size of the reference raster
cell_size = arcpy.GetRasterProperties_management(esa_raster, "CELLSIZEX").getOutput(0)
print(cell_size)

#convert line feature to raster using cell size of reference raster
arcpy.conversion.PolylineToRaster(roads, field, output_raster, "MAXIMUM_LENGTH", "", cell_size)

print("Line feature converted to raster successfully using the cell size of the reference raster.")



9.30599049986377
Line feature converted to raster successfully using the cell size of the reference raster.


In [93]:
print(arcpy.env.workspace)

C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb


In [95]:
#2. merge road and ESA rasters

#overwrite existing outputs 
#arcpy.env.overwriteOutput = True
#want to combined the two rasters - esa and roads

esa_raster = r"C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Forest_ExPot.gdb/esa2020_si"
road_raster = "road_rast"

# Perform the mosaic operation
output_raster = arcpy.management.MosaicToNewRaster(
    input_rasters=[road_raster, esa_raster],
    output_location=gdb_workspace,
    raster_dataset_name_with_extension="esa_and_road",
    coordinate_system_for_the_raster=None,
    pixel_type="8_BIT_UNSIGNED",
    cellsize=None,
    number_of_bands=1,
    mosaic_method="MAXIMUM",
    mosaic_colormap_mode="MATCH"
)

# Print the full path of the saved raster dataset
print("Raster saved to:", output_raster)

# I have no idea why, but instead of the road getting added to the esa layer as 100, it gets added as 18??
# *BE AWARE ROAD = 18 in new mosaic raster*

#note - if you get an error, you need to run checkoutextension. this is if you have a concurrent license  
#if you need to rerun this, need to delete the previous bc mosaic cannot overwrite existing object.

Raster saved to: C:/Users/jc446202/OneDrive - James Cook University (1)/ACIAR_spatial/modified_dat/Fishing_ExPot.gdb\esa_and_road


In [10]:
#print(output_raster)


<Result 'true'>

In [11]:
#4. Add blank ocean raster to fill the ocean space that ESA doesnt cover

full_raster_output = os.path.join(gdb_workspace, "esa_w_road_full_extent")


with arcpy.EnvManager(extent="MAXOF"):
    out_raster = arcpy.ia.Con(
        in_conditional_raster="esa_and_road",
        in_true_raster_or_constant="esa_and_road",
        in_false_raster_or_constant="blank_rast_ocean",
        where_clause="Value IS NOT NULL"
    )
out_raster.save(full_raster_output)


In [13]:
#3. reclassify esa + road raster to travel time
#need to multiple the values by ten because the raster needs to be an integer

remap = arcpy.sa.RemapValue([ #reclassifies land use to travel time (min per km)
    [ 0, 3],     #reclassify 0 to 3 (ocean)
    [10, 60],    # reclassify 10 to 60 (tree cover)
    [18, 1],     # reclassify 18 to 1 (road)
    [20, 38],    # reclassify 20 to 37.5 (shrubland)
    [30, 38],    # reclassify 30 to 37.5 (grassland)
    [40, 38],    # reclassify 40 to 37.5 (cropland)
    [50, 2],     # reclassify 50 to 2 (built up)
    [60, 48],    # reclassify 60 to 48 (bare/sparse veg)
    [70, 60],    # reclassify 70 to 60 (snow and ice)
    [80, 3],     # reclassify 80 to 3 (permanent water)
    [90, 60],    # reclassify 90 to 60 (herbaceous wetland)
    [95, 60],    # reclassify 95 to 60 (mangrove)
    [100, 1],    # reclassify 100 to 1 (road)
])

    
 #run spatial analyst tool and operation to check if it runs
output_raster = "esa_w_road_travel_cost"
    
#perform reclassification
reclass_raster = arcpy.sa.Reclassify("esa_w_road_full_extent", "Value", remap)
reclass_raster.save(output_raster)
print("reclassification complete")
        


reclassification complete


In [ ]:
#4. calculate least cost path


# Define input data
output_raster = os.path.join(gdb_workspace, "travel_dist_cost")
accumulation_type = "LINEAR"  # Other options: CONSTANT, EQUAL

  
        
# Use CostDistance tool with appropriate parameters
#out_distance_accumulation_raster = arcpy.sa.DistanceAccumulation(  #commenting this out because it takes hours to run
    in_source_data="SI_Cities",
    in_barrier_data=None,
    in_surface_raster=None,
    in_cost_raster="esa_w_road_travel_cost",
    in_vertical_raster=None,
    vertical_factor="BINARY 1 -30 30",
    in_horizontal_raster=None,
    horizontal_factor="BINARY 1 45",
    out_back_direction_raster=None,
    out_source_direction_raster=None,
    out_source_location_raster=None,
    source_initial_accumulation=None,
    source_maximum_accumulation=None,
    source_cost_multiplier=None,
    source_direction="",
    distance_method="PLANAR"
)
out_distance_accumulation_raster.save(output_raster)
    
  
print(f"Distance raster with cost and population weight saved to {output_raster}.")




In [ ]:
#before we scale we want to clip the cost raster so its just around the islands, 
#because really were interested in the cost associated with travel between island to reach larger cities, not way out on the edge of the EEZ which really alters the scaled values

arcpy.analysis.Buffer(country_polygon, "Sols_buff200k", 
    buffer_distance_or_field="200 Kilometers",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)

#and make buffer areas just over water areas, remove land
arcpy.analysis.Erase(
    in_features="Sols_buff200k",
    erase_features= country_polygon,
    out_feature_class="Sols_buff_200km_noland",
    cluster_tolerance=None
)

In [ ]:
#clip the travel cost raster to the 200 km buffer
#want land included here, bc presuming distance crossed along land

out_raster = arcpy.sa.ExtractByMask(
    in_raster="travel_dist_cost",
    in_mask_data="Sols_buff200k",
    extraction_area="INSIDE"
)
out_raster.save("travel_dist_cost_200kmbuff")

In [5]:
#scale values 0-100 where near to cities is highest value (greatest potential impact)

out_raster = arcpy.sa.RescaleByFunction(
    in_raster="travel_dist_cost_200kmbuff",
    transformation_function="LINEAR # # # # # #",
    from_scale=100, #this is inverting the rescale so the higher values (100) are acutally closer to the cities.
    to_scale=1
)
out_raster.save("travel_dist_cost_scale")

In [ ]:
# extract by mask so its only in water areas. 

out_raster = arcpy.sa.ExtractByMask(
    in_raster="travel_cost_cities_scale",
    in_mask_data="Sols_buff_200km_noland",
    extraction_area="INSIDE"
)
out_raster.save("travel_dist_cost_scale_noland")



In [4]:
#delete intermediate layers

arcpy.management.Delete("travel_dist_cost_200kmbuff")
arcpy.management.Delete("esa_w_road_travel_cost")
arcpy.management.Delete('esa_w_road_full')
arcpy.management.Delete('road_rast')


<Result 'true'>

<a id= "combineaccess"></a>

## 4d. combine access layers and scale

combining dist_from_coast_scale_full + ocean_pop_5k_scale + travel_dist_cost_scale_no_land
rescaling combined raster



In [6]:
out_raster = arcpy.ia.CellStatistics("dist_from_coast_scale_SI;ocean_pop_10k_scale;travel_dist_cost_scale_noland","SUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT");
out_raster.save("fish_access_layer")

In [7]:
#scale access layer


#identify min and max values of access layer

access_min_output = arcpy.management.GetRasterProperties('fish_access_layer', 'MINIMUM',"") #extracts minimum value from raster
access_min = float(access_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(access_min)

access_max_ouput = arcpy.management.GetRasterProperties('fish_access_layer', 'MAXIMUM',"")
access_max = float(access_max_ouput.getOutput(0))
print(access_max)



0.0
298.452209472656


In [8]:
outraster = ((arcpy.Raster("fish_access_layer") - access_min)/(access_max - access_min))*100; outraster.save("fish_access_scale")


<a id= "impact"></a>


# 7. Combine access, and resource layers

In [9]:
#combine resource and access layers

out_raster = arcpy.ia.CellStatistics("fish_resource_scale;fish_access_scale","SUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT");
out_raster.save("fish_ExPot")

In [11]:
#scale fish impact layer

#identify min and max values of access layer

fish_impact_min_output = arcpy.management.GetRasterProperties('fish_ExPot', 'MINIMUM',"") #extracts minimum value from raster
fish_impact_min = int(fish_impact_min_output.getOutput(0)) #assigns that value to an object, and converts to a integer
print(fish_impact_min)

fish_impact_max_ouput = arcpy.management.GetRasterProperties('fish_ExPot', 'MAXIMUM',"")
fish_impact_max = float(fish_impact_max_ouput.getOutput(0))
print(fish_impact_max)



0
187.440643310547


In [12]:
#scale fishing impact layer
outraster = ((arcpy.Raster("fish_ExPot") - fish_impact_min)/(fish_impact_max - fish_impact_min))*100; outraster.save("fish_ExPot_scale")

# END OF CODE

In [14]:
arcpy.CheckInExtension("Spatial")

'CheckedIn'

In [1]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("report-template.html")
template_variables = {
    "title": item.title,
    "statistics": [s.attributes for s in statistics],
    "features": trees}
generated_html = template.render(template_variables)

TemplateNotFound: report-template.html